In [1]:
import torch

In [2]:
# 若requires_grad=True,则数据类型必须为浮点数类型
w = torch.tensor([1.], requires_grad=True) # backward方法需要指定requires_grad=True(默认requires_grad=False)
x = torch.tensor([2.])

a = w + x
b = torch.add(w, 1)

y = torch.mul(a, b)
y.backward() # 标量对标量求导

print(w)
print(x)
print(a)
print(b)
print(y, end='\n\n')

print(w.is_leaf, x.is_leaf, # w,x为叶tensor
      a.is_leaf,
      b.is_leaf,
      y.is_leaf, end='\n\n')

print(w.requires_grad,
      x.requires_grad,
      a.requires_grad,
      b.requires_grad,
      y.requires_grad, end='\n\n') # 是否参与梯度运算

print(w.grad) # y关于w的梯度
print(x.grad) # x.requires_grad=False,故x.grad=None
print(a.grad) # 仅输出叶tensor .grad属性
print(b.grad)
print(y.grad, end='\n\n')

print(w.grad_fn, # 叶tensor grad_fn属性为None
      x.grad_fn,
      a.grad_fn,
      b.grad_fn,
      y.grad_fn)

tensor([1.], requires_grad=True)
tensor([2.])
tensor([3.], grad_fn=<AddBackward0>)
tensor([2.], grad_fn=<AddBackward0>)
tensor([6.], grad_fn=<MulBackward0>)

True True False False False

True False True True True

tensor([5.])
None
None
None
None

None None <AddBackward0 object at 0x000001E6628490C8> <AddBackward0 object at 0x000001E662849348> <MulBackward0 object at 0x000001E6628492C8>


In [3]:
w = torch.tensor([1.], requires_grad=True)
w.requires_grad_(False) # 修改w的requires_grad属性
print(w.requires_grad, end='\n\n')

x = torch.tensor([2.])
print(x.requires_grad)
x.requires_grad_(True) # 修改x的requires_grad属性
print(x.requires_grad, end='\n\n')

a = w + x
a.retain_grad() # Enables .grad attribute for non-leaf Tensors
b = torch.add(w, 1)
b.retain_grad()

y = torch.mul(a, b)
y.retain_grad()
y.backward()

print(w.grad)
print(x.grad)
print(a.grad) # 通过a.retain_grad(),则可输出y关于a(非叶tensor)的梯度
print(b.grad)
print(y.grad)

False

False
True

None
tensor([2.])
tensor([2.])
None
tensor([1.])


In [4]:
x = torch.tensor([2.], requires_grad=True)
y = x**4

y.backward(retain_graph=True) # If False, the graph used to compute the grads will be freed
print(x.grad)

y.backward(retain_graph=True)
print(x.grad) # 梯度累加

y.backward()
print(x.grad) # 梯度继续累加

y.backward() # 报错,此时计算图已被释放

tensor([32.])
tensor([64.])
tensor([96.])


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.